In [ ]:
import tensorflow as tf
import numpy as np
import math
%matplotlib inline
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.cross_validation import train_test_split
from tensorflow.models.image.cifar10 import cifar10_input
from keras.datasets import cifar10
import datetime

def lrelu(x, leak=0.2, name="lrelu"):
    with tf.variable_scope(name):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
        return f1 * x + f2 * abs(x)
    
def autoencoder(input_shape=[None, 32, 32, 3],
                n_filters=[3, 3, 3, 3],
                filter_sizes=[5, 5, 5],
                corruption=False):
    # %%
    # input to the network
    x = tf.placeholder(
        tf.float32, input_shape, name='x')


    # %%
    # ensure 2-d is converted to square tensor.
    if len(x.get_shape()) == 2:
        x_dim = np.sqrt(x.get_shape().as_list()[1])
        if x_dim != int(x_dim):
            raise ValueError('Unsupported input dimensions')
        x_dim = int(x_dim)
        x_tensor = tf.reshape(
            x, [-1, x_dim, x_dim, n_filters[0]])
    elif len(x.get_shape()) == 4:
        x_tensor = x
    else:
        raise ValueError('Unsupported input dimensions')
    current_input = x_tensor

    # %%
    # Build the encoder
    encoder_weights = []
    encoder_ops = []
    shapes = []
    for layer_i, n_output in enumerate(n_filters[1:]):
        n_input = current_input.get_shape().as_list()[3]
        shapes.append(current_input.get_shape().as_list())
        W = tf.Variable(
            tf.random_uniform([
                filter_sizes[layer_i],
                filter_sizes[layer_i],
                n_input, n_output],
                -1.0 / math.sqrt(n_input),
                1.0 / math.sqrt(n_input)))
        b = tf.Variable(tf.zeros([n_output]))
        encoder_weights.append(W)
        output = tf.nn.sigmoid(
            tf.add(tf.nn.conv2d(
                current_input, W, strides=[1, 1, 1, 1], padding='SAME'), b))
        encoder_ops.append(output)
        current_input = output

    # %%
    # store the latent representation
    z = current_input
    encoder_weights.reverse()
    shapes.reverse()

    # %%
    # Build the decoder using the same weights
    for layer_i, shape in enumerate(shapes):
        W = encoder_weights[layer_i]
        b = tf.Variable(tf.zeros([W.get_shape().as_list()[2]]))
        output = tf.nn.sigmoid(
            tf.add(tf.nn.conv2d_transpose(
                current_input, W,
                tf.pack([tf.shape(x)[0], shape[1], shape[2], shape[3]]),
                strides=[1, 1, 1, 1], padding='SAME'), b))
        current_input = output
        
    decoder = current_input

    # %%
    # now have the reconstruction through the network
    y = current_input
    # cost function measures pixel-wise difference
    cost = tf.reduce_sum(tf.square(y - x_tensor))

    # %%
    return {'x': x, 'z': z, 'y': y, 'cost': cost, 
            "encoder": encoder_ops, "decoder": decoder}


# %%
def test_cifar(n_filters, filter_sizes):
    """Test the convolutional autoencder using MNIST."""
    # %%
    import tensorflow as tf
    from keras.datasets import cifar10
    import matplotlib.pyplot as plt

    # %%
    # load CIFAR10
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train = X_train / 255.
    X_test = X_test / 255.
    ae = autoencoder(n_filters=n_filters, filter_sizes=filter_sizes)

    # %%
    learning_rate = 0.01
    optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(ae['cost'])

    # %%
    # We create a session to use the graph
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.run(tf.initialize_all_variables())

    # %%
    # Fit all training data
    batch_size = 100
    n_epochs = 100
    step_size = 10
    for epoch_i in range(n_epochs):
        for batch_i in range(X_train.shape[0] // batch_size):
            batch_xs = X_train[batch_i * batch_size:(batch_i + 1) * batch_size]
            train = batch_xs
            sess.run(optimizer, feed_dict={ae['x']: train})
        if epoch_i % step_size == 0:
            print(str(datetime.datetime.now()), epoch_i, sess.run(ae['cost'], feed_dict={ae['x']: train}))

    # %%
    # Plot example reconstructions
    n_examples = 10
    test_xs = X_test[:n_examples]
    test_xs_norm = test_xs
    recon = sess.run(ae['y'], feed_dict={ae['x']: test_xs_norm})
    print(recon.shape)
    fig, axs = plt.subplots(2, n_examples, figsize=(10, 2))
    for example_i in range(n_examples):
        axs[0][example_i].imshow(
            np.reshape(test_xs[example_i, :], (32, 32, 3)))
        axs[0][example_i].axis("off")

        axs[1][example_i].imshow(
            np.reshape(
                np.reshape(recon[example_i, ...], (32, 32, 3)),
                (32, 32, 3)))
        axs[1][example_i].axis("off")
    
    fig.show()
    plt.draw()
    
    ae["session"] = sess
    
    return ae

In [ ]:
# ae = test_cifar(n_filters=[3, 20, 20, 20, 20, 20], filter_sizes=[3, 3, 3, 3, 3])
ae = test_cifar(n_filters=[3,10,10,10], filter_sizes=[3,3,3])

In [ ]:
num_examples = 10000

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train / 255.
X_test = X_test / 255.
# mean_img = np.mean(X_train, axis=0)
# train = X_train[:num_examples] - mean_img
train = X_train[:num_examples]
y_train = y_train.ravel()
y_test = y_test.ravel()

In [ ]:
sess = ae["session"]
layers = [sess.run(ae["encoder"][i], 
        feed_dict={ae['x']: train}) for i in range(len(ae["encoder"]))]

In [ ]:
ravels = (np.array([row.ravel() for row in layers[i]]) for i in range(len(ae["encoder"])))
combined = np.hstack(ravels)

In [ ]:
combined.shape

In [ ]:
# # Scale and visualize the embedding vectors
# def plot_embedding(X, y, title=None):
#     x_min, x_max = np.min(X, 0), np.max(X, 0)
#     X = (X - x_min) / (x_max - x_min)

#     plt.figure(figsize=(20, 10))
#     ax = plt.subplot(111)
#     for i in range(X.shape[0]):
#         plt.text(X[i, 0], X[i, 1], str(y[i]),
#                  color=plt.cm.Set1(y[i] / 10.),
#                  fontdict={'weight': 'bold', 'size': 12})

#     plt.xticks([]), plt.yticks([])
#     if title is not None:
#         plt.title(title)

# vectorized_imgs = combined

In [ ]:
from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, random_state=0)
# np.set_printoptions(suppress=True)
# X_tsne = tsne.fit_transform(vectorized_imgs)

In [ ]:
y = list(y_train)[:10000]
print str(datetime.datetime.now())
# plot_embedding(X_tsne, y)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=3)
# knn.fit(X_tsne, y)

In [ ]:
# knn.score(X_tsne, y)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot_embedding_3d(X, y, title=None):
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(111, projection='3d')    
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    for i in range(X.shape[0]):
        ax.text(X[i, 0], X[i, 1], X[i, 2], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'size': 5})
    ax.set_zticks([])
    ax.set_xticks([]), ax.set_yticks([])

    if title is not None:
        plt.title(title)

In [ ]:
tsne = TSNE(n_components=3, random_state=0)
np.set_printoptions(suppress=True)
X_tsne_3 = tsne.fit_transform(combined)

In [ ]:
plot_embedding_3d(X_tsne_3, y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
print knn.fit(combined, y)
print str(datetime.datetime.now())

In [ ]:
print knn.score(combined, y)
print str(datetime.datetime.now())

In [ ]:
ks = [3, 5, 10, 20]

from sklearn.neighbors import KNeighborsClassifier

for k in ks:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(combined, y)
    print ("KNN Score with k=%i" % k,  knn.score(combined, y))

In [ ]:
y_pred = knn.predict(combined)
print str(datetime.datetime.now())

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2)
print knn.fit(combined, y)
print str(datetime.datetime.now())

In [ ]:
y_pred = knn.predict(combined)
print str(datetime.datetime.now())

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np

plt.figure(figsize=(30, 30))

class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
#     plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
#     plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, str(cm[i, j])[:4],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cnf_matrix = confusion_matrix(y, y_pred)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix for CIFAR-10')

plt.show()

In [ ]:
# from scipy.spatial import cKDTree
# import random

# test_idxes = random.sample(range(combined.shape[0]), 10)
# print test_idxes

# tree = cKDTree(combined)
# test_array = combined[test_idxes]
# query_res = tree.query(test_array, k=5)

In [ ]:
# nns = []

# for idx, row in enumerate(query_res[1]):
#     nn = X_test[row.ravel()]
# #     to_plot = np.vstack([X_test[idx], nn])
#     nns.append(nn)
    
# nns = np.stack(nns)

In [ ]:
# def plot_nearest_neighbors(nns, reshape=(28, 28)):
#     fig, axs = plt.subplots(nns.shape[0], nns.shape[1], figsize=(32, 32))
    
#     for i in range(nns.shape[0]):
#         for j in range(nns.shape[1]):
#             if len(reshape) != 2:
#                 axs[i][j].imshow(
#                     nns[i, j].reshape(reshape))
#             else:
#                 axs[i][j].imshow(
#                     nns[i, j].reshape(reshape), cmap="Greys_r")
#             axs[i][j].axis("off")
#     fig.subplots_adjust(wspace=0, hspace=0)
#     fig.tight_layout()
    
# plot_nearest_neighbors(nns, reshape=(32, 32, 3))